# Preprocessing Data Komentar YouTube

Notebook ini melakukan preprocessing data komentar dengan langkah-langkah:
1. Filter Konteks (Strategi KETAT)
2. Bersihkan Teks (URL, Unicode, Emoji, lowercase, tanda baca, angka)
3. Tokenization
4. Hapus Stopwords
5. Gabungkan Kembali

In [21]:
# Import libraries
import pandas as pd
import re
import unicodedata
import demoji

# Download emoji data (run once)
demoji.download_codes()

C:\Users\rhzain\AppData\Local\Temp\ipykernel_38252\1846252205.py:8: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [22]:
# Load data
df = pd.read_csv('raw-scrape-yt.csv')
print(f"Total komentar: {len(df)}")
print("\nContoh data awal:")
df.head()

Total komentar: 1196

Contoh data awal:


,Video_ID,Teks_Komentar
0,MIo4tGN11j0,"berdoa tidak ada comment judol, amin."
1,MIo4tGN11j0,kereen.
2,MIo4tGN11j0,"Sempat mikir mau pindah ke negara sebelah, nge..."
3,MIo4tGN11j0,"Kalo kabur mau kemana ke Singapur ,emang di Si..."
4,MIo4tGN11j0,Jg ke malaysia.....pergi sono india bangla nep...


## 1. Filter Konteks - Strategi KETAT

Membuang komentar noise yang tidak relevan

In [23]:
def filter_konteks_ketat(teks):
    """
    Filter komentar noise dengan strategi KETAT.
    Return True jika komentar valid, False jika noise.
    """
    if pd.isna(teks) or len(teks.strip()) < 3:
        return False
    
    teks_lower = teks.lower().strip()
    
    # Daftar pola noise yang harus dibuang
    noise_patterns = [
        r'^[@#]',  # Dimulai dengan @ atau #
        r'^\d+$',  # Hanya angka
        r'^[^\w\s]+$',  # Hanya simbol/emoji
        r'(subscribe|subs|subrek|gw klik subs)',  # Promosi subscribe
        r'(like|liek|liak|laik)\s*(dulu|dong|yuk)',  # Ajakan like
        r'(pin|ping|pin dong|pinn)',  # Minta pin
        r'^(wkwk|haha|xixi|kwkw|wkwkwk|hahaha)+$',  # Hanya ketawa
        r'^(amin|amiin|aamiin)+$',  # Hanya amin
        r'^(pertamax|first|kedua|ketiga)',  # Klaim urutan
        r'(giveaway|give away|kontes)',  # Promo
        r'^(yang \d{4})',  # "yang 2024", "yang 2025", dst
        r'(judol|judi online)', # Filter judi online
    ]
    
    for pattern in noise_patterns:
        if re.search(pattern, teks_lower):
            return False
    
    # Komentar terlalu pendek (< 10 karakter setelah strip)
    if len(teks_lower) < 10:
        return False
    
    return True

# Terapkan filter
df['is_valid'] = df['Teks_Komentar'].apply(filter_konteks_ketat)
df_filtered = df[df['is_valid']].copy()
df_filtered = df_filtered.drop('is_valid', axis=1)

print(f"Komentar sebelum filter: {len(df)}")
print(f"Komentar setelah filter: {len(df_filtered)}")
print(f"Komentar terbuang: {len(df) - len(df_filtered)}")

Komentar sebelum filter: 1196
Komentar setelah filter: 1060
Komentar terbuang: 136


## 2. Bersihkan Teks

Membersihkan teks dari URL, normalisasi Unicode, konversi emoji, lowercase, tanda baca, dan angka

In [24]:
def bersihkan_teks(teks):
    """
    Membersihkan teks dengan langkah-langkah:
    1. Hapus URL
    2. Normalisasi Unicode
    3. Ganti Emoji ke Teks
    4. Lowercase
    5. Hapus tanda baca dan angka
    """
    if pd.isna(teks):
        return ""
    
    # 1. Hapus URL
    teks = re.sub(r'http\S+|www\.\S+', '', teks)
    
    # 2. Normalisasi Unicode (NFD -> hapus combining characters -> NFC)
    teks = unicodedata.normalize('NFD', teks)
    teks = ''.join(char for char in teks if unicodedata.category(char) != 'Mn')
    teks = unicodedata.normalize('NFC', teks)
    
    # 3. Ganti Emoji ke Teks
    teks = demoji.replace_with_desc(teks, sep=" ")
    
    # 4. Lowercase
    teks = teks.lower()
    
    # 5. Hapus tanda baca dan angka
    # Hanya simpan huruf dan spasi
    teks = re.sub(r'[^a-z\s]', '', teks)
    
    # Hapus spasi berlebih
    teks = re.sub(r'\s+', ' ', teks).strip()
    
    return teks

# Terapkan pembersihan
df_filtered['teks_bersih'] = df_filtered['Teks_Komentar'].apply(bersihkan_teks)

print("Contoh hasil pembersihan:")
print("\nSebelum:")
print(df_filtered['Teks_Komentar'].iloc[0])
print("\nSesudah:")
print(df_filtered['teks_bersih'].iloc[0])

Contoh hasil pembersihan:

Sebelum:
Kalo kabur mau kemana ke Singapur ,emang di Singapur tinggal dimana rakyat Singapur aja nggak punya rumah mereka masih nyicil😂"katanya negara kaya Singapur "

Sesudah:
kalo kabur mau kemana ke singapur emang di singapur tinggal dimana rakyat singapur aja nggak punya rumah mereka masih nyicil face with tears of joy katanya negara kaya singapur


## 3. Tokenization

Mengubah kalimat menjadi daftar kata-kata

In [25]:
def tokenize(teks):
    """
    Tokenize teks menjadi list kata-kata menggunakan split()
    """
    if pd.isna(teks) or len(teks.strip()) == 0:
        return []
    return teks.split()

# Terapkan tokenization
df_filtered['tokens'] = df_filtered['teks_bersih'].apply(tokenize)

print("Contoh hasil tokenization:")
print(f"Teks: {df_filtered['teks_bersih'].iloc[0]}")
print(f"Tokens: {df_filtered['tokens'].iloc[0]}")

Contoh hasil tokenization:
Teks: kalo kabur mau kemana ke singapur emang di singapur tinggal dimana rakyat singapur aja nggak punya rumah mereka masih nyicil face with tears of joy katanya negara kaya singapur
Tokens: ['kalo', 'kabur', 'mau', 'kemana', 'ke', 'singapur', 'emang', 'di', 'singapur', 'tinggal', 'dimana', 'rakyat', 'singapur', 'aja', 'nggak', 'punya', 'rumah', 'mereka', 'masih', 'nyicil', 'face', 'with', 'tears', 'of', 'joy', 'katanya', 'negara', 'kaya', 'singapur']


## 4. Hapus Stopwords

Membuat daftar stopwords gabungan (Indonesia + Inggris) dan membuang kata-kata tersebut

In [26]:
# Daftar stopwords Indonesia
stopwords_id = set([
    'ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir',
    'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara',
    'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan',
    'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana',
    'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik',
    'bakal', 'bakalan', 'balik', 'banyak', 'banyaknya', 'bapak', 'baru', 'bawah', 'beberapa',
    'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun',
    'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah',
    'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun',
    'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya',
    'berjumlah', 'berkali', 'berkehendak', 'berkenaan', 'berlainan', 'berlalu', 'berlangsung',
    'berlebihan', 'bermacam', 'bermaksud', 'bermula', 'bersama', 'bersama-sama', 'bersiap', 'bersiap-siap',
    'bertanya', 'bertanya-tanya', 'berturut', 'berturut-turut', 'bertutur', 'berujar', 'berupa',
    'besar', 'betul', 'betulkah', 'biasa', 'biasanya', 'bila', 'bilakah', 'bisa', 'bisakah',
    'boleh', 'bolehkah', 'bolehlah', 'buat', 'bukan', 'bukankah', 'bukanlah', 'bukannya', 'bulan',
    'bung', 'cara', 'caranya', 'cukup', 'cukupkah', 'cukuplah', 'cuma', 'dahulu', 'dalam', 'dan',
    'dapat', 'dari', 'daripada', 'datang', 'dekat', 'demi', 'demikian', 'demikianlah', 'dengan',
    'depan', 'di', 'dia', 'diakhiri', 'diakhirinya', 'dialah', 'diantara', 'diantaranya', 'diberi',
    'diberikan', 'diberikannya', 'dibuat', 'dibuatnya', 'didapat', 'didatangkan', 'digunakan', 'diibaratkan',
    'diibaratkannya', 'diingat', 'diingatkan', 'diinginkan', 'dijawab', 'dijelaskan', 'dijelaskannya',
    'dikarenakan', 'dikatakan', 'dikatakannya', 'dikehendaki', 'dikenakan', 'dikerjakan', 'diketahui',
    'diketahuinya', 'dikira', 'dilakukan', 'dilalui', 'dilihat', 'dimaksud', 'dimaksudkan', 'dimaksudkannya',
    'dimaksudnya', 'diminta', 'dimintai', 'dimisalkan', 'dimulai', 'dimulailah', 'dimulainya', 'dimungkinkan',
    'dini', 'dipastikan', 'diperbuat', 'diperbuatnya', 'dipergunakan', 'diperkirakan', 'diperlihatkan',
    'diperlukan', 'diperlukannya', 'dipersoalkan', 'dipertanyakan', 'dipunyai', 'diri', 'dirinya',
    'disampaikan', 'disebut', 'disebutkan', 'disebutkannya', 'disini', 'disinilah', 'ditambahkan',
    'ditandaskan', 'ditanya', 'ditanyai', 'ditanyakan', 'ditegaskan', 'ditujukan', 'ditunjuk', 'ditunjuki',
    'ditunjukkan', 'ditunjukkannya', 'ditunjuknya', 'dituturkan', 'dituturkannya', 'diucapkan', 'diucapkannya',
    'diungkapkan', 'dong', 'dua', 'dulu', 'empat', 'enggak', 'enggaknya', 'entah', 'entahlah', 'guna',
    'gunakan', 'hal', 'hampir', 'hanya', 'hanyalah', 'hari', 'harus', 'haruslah', 'harusnya', 'hendak',
    'hendaklah', 'hendaknya', 'hingga', 'ia', 'ialah', 'ibarat', 'ibaratkan', 'ibaratnya', 'ibu',
    'ikut', 'ingat', 'ingat-ingat', 'ingin', 'inginkah', 'inginkan', 'ini', 'inikah', 'inilah',
    'itu', 'itukah', 'itulah', 'jadi', 'jadilah', 'jadinya', 'jangan', 'jangankan', 'janganlah',
    'jauh', 'jawab', 'jawaban', 'jawabnya', 'jelas', 'jelaskan', 'jelaslah', 'jelasnya', 'jika',
    'jikalau', 'juga', 'jumlah', 'jumlahnya', 'justru', 'kala', 'kalau', 'kalaulah', 'kalaupun',
    'kalian', 'kami', 'kamilah', 'kamu', 'kamulah', 'kan', 'kapan', 'kapankah', 'kapanpun', 'karena',
    'karenanya', 'kasus', 'kata', 'katakan', 'katakanlah', 'katanya', 'ke', 'keadaan', 'kebetulan',
    'kecil', 'kedua', 'keduanya', 'keinginan', 'kelamaan', 'kelihatan', 'kelihatannya', 'kelima',
    'keluar', 'kembali', 'kemudian', 'kemungkinan', 'kemungkinannya', 'kenapa', 'kepada', 'kepadanya',
    'kesampaian', 'keseluruhan', 'keseluruhannya', 'keterlaluan', 'ketika', 'khususnya', 'kini',
    'kinilah', 'kira', 'kira-kira', 'kiranya', 'kita', 'kitalah', 'kok', 'kurang', 'lagi', 'lagian',
    'lah', 'lain', 'lainnya', 'lalu', 'lama', 'lamanya', 'langsung', 'lebih', 'lewat', 'lima',
    'luar', 'lusa', 'maka', 'makanya', 'makin', 'malah', 'malahan', 'mampu', 'mampukah', 'mana',
    'manakala', 'manalagi', 'masa', 'masalah', 'masalahnya', 'masih', 'masihkah', 'masing', 'masing-masing',
    'mau', 'maupun', 'melainkan', 'melakukan', 'melalui', 'melihat', 'melihatnya', 'memang', 'memastikan',
    'memberi', 'memberikan', 'membuat', 'memerlukan', 'memihak', 'meminta', 'memintakan', 'memisalkan',
    'memperbuat', 'mempergunakan', 'memperkirakan', 'memperlihatkan', 'mempersiapkan', 'mempersoalkan',
    'mempertanyakan', 'mempunyai', 'memulai', 'memungkinkan', 'menaiki', 'menambahkan', 'menandaskan',
    'menanti', 'menanti-nanti', 'menantikan', 'menanya', 'menanyai', 'menanyakan', 'mendapat', 'mendapatkan',
    'mendatang', 'mendatangi', 'mendatangkan', 'menegaskan', 'mengakhiri', 'mengapa', 'mengatakan',
    'mengatakannya', 'mengenai', 'mengerjakan', 'mengetahui', 'menggunakan', 'menghendaki', 'mengibaratkan',
    'mengibaratkannya', 'mengingat', 'mengingatkan', 'menginginkan', 'mengira', 'mengucapkan', 'mengucapkannya',
    'mengungkapkan', 'menjadi', 'menjawab', 'menjelaskan', 'menuju', 'menunjuk', 'menunjuki', 'menunjukkan',
    'menunjuknya', 'menurut', 'menuturkan', 'menyampaikan', 'menyangkut', 'menyatakan', 'menyebutkan',
    'menyeluruh', 'menyiapkan', 'merasa', 'mereka', 'merekalah', 'merupakan', 'meski', 'meskipun',
    'meyakini', 'meyakinkan', 'minta', 'mirip', 'misal', 'misalkan', 'misalnya', 'mula', 'mulai',
    'mulailah', 'mulanya', 'mungkin', 'mungkinkah', 'nah', 'naik', 'namun', 'nanti', 'nantinya',
    'nyaris', 'nyatanya', 'oleh', 'olehnya', 'pada', 'padahal', 'padanya', 'pak', 'paling', 'panjang',
    'pantas', 'para', 'pasti', 'pastilah', 'penting', 'pentingnya', 'per', 'percuma', 'perlu',
    'perlukah', 'perlunya', 'pernah', 'persoalan', 'pertama', 'pertama-tama', 'pertanyaan', 'pertanyakan',
    'pihak', 'pihaknya', 'pukul', 'pula', 'pun', 'punya', 'rasa', 'rasanya', 'rata', 'rupanya',
    'saat', 'saatnya', 'saja', 'sajalah', 'saling', 'sama', 'sama-sama', 'sambil', 'sampai', 'sampai-sampai',
    'sampaikan', 'sana', 'sangat', 'sangatlah', 'satu', 'saya', 'sayalah', 'se', 'sebab', 'sebabnya',
    'sebagai', 'sebagaimana', 'sebagainya', 'sebagian', 'sebaik', 'sebaik-baiknya', 'sebaiknya', 'sebaliknya',
    'sebanyak', 'sebegini', 'sebegitu', 'sebelum', 'sebelumnya', 'sebenarnya', 'seberapa', 'sebesar',
    'sebetulnya', 'sebisanya', 'sebuah', 'sebut', 'sebutlah', 'sebutnya', 'secara', 'secukupnya',
    'sedang', 'sedangkan', 'sedemikian', 'sedikit', 'sedikitnya', 'seenaknya', 'segala', 'segalanya',
    'segera', 'seharusnya', 'sehingga', 'seingat', 'sejak', 'sejauh', 'sejenak', 'sejumlah', 'sekadar',
    'sekadarnya', 'sekali', 'sekali-kali', 'sekalian', 'sekaligus', 'sekalipun', 'sekarang', 'sekarang',
    'sekecil', 'seketika', 'sekiranya', 'sekitar', 'sekitarnya', 'sekurang-kurangnya', 'sekurangnya',
    'sela', 'selain', 'selaku', 'selalu', 'selama', 'selama-lamanya', 'selamanya', 'selanjutnya',
    'seluruh', 'seluruhnya', 'semacam', 'semakin', 'semampu', 'semampunya', 'semasa', 'semasih',
    'semata', 'semata-mata', 'semaunya', 'sementara', 'semisal', 'semisalnya', 'sempat', 'semua',
    'semuanya', 'semula', 'sendiri', 'sendirian', 'sendirinya', 'seolah', 'seolah-olah', 'seorang',
    'sepanjang', 'sepantasnya', 'sepantasnyalah', 'seperlunya', 'seperti', 'sepertinya', 'sepihak',
    'sering', 'seringnya', 'serta', 'serupa', 'sesaat', 'sesama', 'sesampai', 'sesegera', 'sesekali',
    'seseorang', 'sesuatu', 'sesuatunya', 'sesudah', 'sesudahnya', 'setelah', 'setelahnya', 'setempat',
    'setengah', 'seterusnya', 'setiap', 'setiba', 'setibanya', 'setidak-tidaknya', 'setidaknya',
    'setinggi', 'sewajarnya', 'sewaktu', 'si', 'sia', 'siap', 'siapa', 'siapakah', 'siapapun',
    'sini', 'sinilah', 'soal', 'soalnya', 'suatu', 'sudah', 'sudahkah', 'sudahlah', 'supaya', 'tadi',
    'tadinya', 'tahu', 'tahun', 'tak', 'tambah', 'tambahnya', 'tampak', 'tampaknya', 'tandas', 'tandasnya',
    'tanpa', 'tanya', 'tanyakan', 'tanyanya', 'tapi', 'tegas', 'tegasnya', 'telah', 'tempat', 'tengah',
    'tentang', 'tentu', 'tentulah', 'tentunya', 'tepat', 'terakhir', 'terasa', 'terbanyak', 'terdahulu',
    'terdapat', 'terdiri', 'terhadap', 'terhadapnya', 'teringat', 'teringat-ingat', 'terjadi', 'terjadilah',
    'terjadinya', 'terkira', 'terlalu', 'terlebih', 'terlihat', 'termasuk', 'ternyata', 'tersampaikan',
    'tersebut', 'tersebutlah', 'tertentu', 'tertuju', 'terus', 'terutama', 'tetap', 'tetapi', 'tiap',
    'tiba', 'tiba-tiba', 'tidak', 'tidakkah', 'tidaklah', 'tiga', 'tinggi', 'toh', 'tunjuk', 'turut',
    'tutur', 'tuturnya', 'ucap', 'ucapnya', 'ujar', 'ujarnya', 'umum', 'umumnya', 'ungkap', 'ungkapnya',
    'untuk', 'usah', 'usai', 'waduh', 'wah', 'wahai', 'waktu', 'waktunya', 'walau', 'walaupun',
    'wong', 'yaitu', 'yakin', 'yakni', 'yang'
])

# Daftar stopwords Inggris
stopwords_en = set([
    'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are',
    'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but',
    'by', 'can', 'could', 'did', 'do', 'does', 'doing', 'down', 'during', 'each', 'few', 'for',
    'from', 'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here', 'hers', 'herself',
    'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'it', 'its', 'itself', 'just',
    'me', 'might', 'more', 'most', 'must', 'my', 'myself', 'no', 'nor', 'not', 'now', 'of', 'off',
    'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 's',
    'same', 'she', 'should', 'so', 'some', 'such', 't', 'than', 'that', 'the', 'their', 'theirs',
    'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too',
    'under', 'until', 'up', 'very', 'was', 'we', 'were', 'what', 'when', 'where', 'which', 'while',
    'who', 'whom', 'why', 'will', 'with', 'would', 'you', 'your', 'yours', 'yourself', 'yourselves'
])

# Gabungkan stopwords
stopwords = stopwords_id.union(stopwords_en)

print(f"Total stopwords: {len(stopwords)}")
print(f"Stopwords Indonesia: {len(stopwords_id)}")
print(f"Stopwords Inggris: {len(stopwords_en)}")

Total stopwords: 889
Stopwords Indonesia: 759
Stopwords Inggris: 130


In [27]:
def hapus_stopwords(tokens):
    """
    Hapus stopwords dari list tokens
    """
    return [word for word in tokens if word not in stopwords and len(word) > 1]

# Terapkan penghapusan stopwords
df_filtered['tokens_filtered'] = df_filtered['tokens'].apply(hapus_stopwords)

print("Contoh hasil penghapusan stopwords:")
print(f"Sebelum: {df_filtered['tokens'].iloc[0]}")
print(f"Sesudah: {df_filtered['tokens_filtered'].iloc[0]}")

Contoh hasil penghapusan stopwords:
Sebelum: ['kalo', 'kabur', 'mau', 'kemana', 'ke', 'singapur', 'emang', 'di', 'singapur', 'tinggal', 'dimana', 'rakyat', 'singapur', 'aja', 'nggak', 'punya', 'rumah', 'mereka', 'masih', 'nyicil', 'face', 'with', 'tears', 'of', 'joy', 'katanya', 'negara', 'kaya', 'singapur']
Sesudah: ['kalo', 'kabur', 'kemana', 'singapur', 'emang', 'singapur', 'tinggal', 'dimana', 'rakyat', 'singapur', 'aja', 'nggak', 'rumah', 'nyicil', 'face', 'tears', 'joy', 'negara', 'kaya', 'singapur']


## 5. Gabungkan Kembali

Menggabungkan list kata-kata bersih menjadi satu kalimat

In [28]:
def gabungkan_tokens(tokens):
    """
    Gabungkan list tokens menjadi string dengan spasi
    """
    return " ".join(tokens)

# Gabungkan kembali tokens menjadi teks final
df_filtered['teks_final'] = df_filtered['tokens_filtered'].apply(gabungkan_tokens)

# Hapus baris yang teksnya kosong setelah preprocessing
df_final = df_filtered[df_filtered['teks_final'].str.strip() != ''].copy()

print(f"Komentar sebelum preprocessing: {len(df)}")
print(f"Komentar setelah preprocessing lengkap: {len(df_final)}")
print(f"\nContoh hasil akhir preprocessing:")
print("="*80)
for i in range(min(3, len(df_final))):
    print(f"\nAsli: {df_final['Teks_Komentar'].iloc[i][:100]}...")
    print(f"Final: {df_final['teks_final'].iloc[i]}")
    print("-"*80)

Komentar sebelum preprocessing: 1196
Komentar setelah preprocessing lengkap: 1059

Contoh hasil akhir preprocessing:

Asli: Kalo kabur mau kemana ke Singapur ,emang di Singapur tinggal dimana rakyat Singapur aja nggak punya ...
Final: kalo kabur kemana singapur emang singapur tinggal dimana rakyat singapur aja nggak rumah nyicil face tears joy negara kaya singapur
--------------------------------------------------------------------------------

Asli: Jg ke malaysia.....pergi sono india bangla nepal serumpun....rupiah rupie...
Final: jg malaysiapergi sono india bangla nepal serumpunrupiah rupie
--------------------------------------------------------------------------------

Asli: gue mulai demen nih org...
Final: gue demen nih org
--------------------------------------------------------------------------------


## 6. Simpan Hasil

Menyimpan hasil preprocessing ke file CSV

In [29]:
# Simpan data final (teks asli dan teks hasil preprocessing)
output_file = 'data_preprocessed.csv'
df_final[['Teks_Komentar', 'teks_final']].to_csv(output_file, index=False, encoding='utf-8')

print(f"✅ Data berhasil disimpan ke '{output_file}'")
print(f"Total data: {len(df_final)} komentar")
print(f"\nKolom yang tersimpan:")
print("- Teks_Komentar: Teks asli komentar")
print("- teks_final: Teks hasil preprocessing")

✅ Data berhasil disimpan ke 'data_preprocessed.csv'
Total data: 1059 komentar

Kolom yang tersimpan:
- Teks_Komentar: Teks asli komentar
- teks_final: Teks hasil preprocessing


## 7. Statistik Preprocessing (Opsional)

Menampilkan statistik dari proses preprocessing

In [30]:
# Hitung statistik
total_awal = len(df)
total_setelah_filter = len(df_filtered)
total_final = len(df_final)
persentase_valid = (total_final / total_awal * 100) if total_awal > 0 else 0

# Hitung rata-rata panjang teks
avg_len_asli = df_final['Teks_Komentar'].str.len().mean()
avg_len_final = df_final['teks_final'].str.len().mean()

# Hitung rata-rata jumlah kata
avg_words_asli = df_final['Teks_Komentar'].str.split().str.len().mean()
avg_words_final = df_final['teks_final'].str.split().str.len().mean()

print("=" * 60)
print("STATISTIK PREPROCESSING")
print("=" * 60)
print(f"\n📊 Jumlah Data:")
print(f"   - Data awal             : {total_awal:,} komentar")
print(f"   - Setelah filter konteks: {total_setelah_filter:,} komentar")
print(f"   - Data akhir valid      : {total_final:,} komentar")
print(f"   - Persentase data valid : {persentase_valid:.2f}%")
print(f"   - Data terbuang         : {total_awal - total_final:,} komentar")

print(f"\n📝 Rata-rata Panjang Teks (karakter):")
print(f"   - Teks asli  : {avg_len_asli:.2f}")
print(f"   - Teks final : {avg_len_final:.2f}")
print(f"   - Reduksi    : {((avg_len_asli - avg_len_final) / avg_len_asli * 100):.2f}%")

print(f"\n📄 Rata-rata Jumlah Kata:")
print(f"   - Teks asli  : {avg_words_asli:.2f}")
print(f"   - Teks final : {avg_words_final:.2f}")
print(f"   - Reduksi    : {((avg_words_asli - avg_words_final) / avg_words_asli * 100):.2f}%")

print("\n" + "=" * 60)

STATISTIK PREPROCESSING

📊 Jumlah Data:
   - Data awal             : 1,196 komentar
   - Setelah filter konteks: 1,060 komentar
   - Data akhir valid      : 1,059 komentar
   - Persentase data valid : 88.55%
   - Data terbuang         : 137 komentar

📝 Rata-rata Panjang Teks (karakter):
   - Teks asli  : 136.54
   - Teks final : 94.15
   - Reduksi    : 31.04%

📄 Rata-rata Jumlah Kata:
   - Teks asli  : 21.29
   - Teks final : 14.02
   - Reduksi    : 34.15%

